In [ ]:
# Loading required libraries:
import numpy as np
import cv2

# Paths for the model files 
protoFile = "D:/Study Material/Data_Science/DIY/Coloring Black & White Images with Deep Learning/model/colorization_deploy_v2.prototxt"
weightsFile = "D:/Study Material/Data_Science/DIY/Coloring Black & White Images with Deep Learning/model/colorization_release_v2.caffemodel"
points = np.load("D:/Study Material/Data_Science/DIY/Coloring Black & White Images with Deep Learning/model/pts_in_hull.npy")

# Read input image:
img = cv2.imread("D:/Study Material/Data_Science/DIY/Coloring Black & White Images with Deep Learning/images/lion.jpg")

W_in = 224
H_in = 224
 
# Read network & bin centers into Memory: 
net = cv2.dnn.readNetFromCaffe(protoFile, weightsFile)
points = points.transpose().reshape(2, 313, 1, 1)
net.getLayer(net.getLayerId('class8_ab')).blobs = [points.astype(np.float32)]
net.getLayer(net.getLayerId('conv8_313_rh')).blobs = [np.full([1, 313], 2.606, np.float32)]

#Convert RGB values of Input image to range of 0 to 1:
img_rgb = (img[:,:,[2, 1, 0]] * 1.0 / 255).astype(np.float32)
img_lab = cv2.cvtColor(img_rgb, cv2.COLOR_RGB2Lab)
img_l = img_lab[:,:,0] # pull out Lightness channel

# Resizing Lightness channel to network input size: 
img_l_rs = cv2.resize(img_l, (W_in, H_in))
img_l_rs -= 50 #Mean-centering

net.setInput(cv2.dnn.blobFromImage(img_l_rs))
result = net.forward()[0,:,:,:].transpose((1,2,0))
 
(H,W) = img_rgb.shape[:2]
result_us = cv2.resize(result, (W,H))

# Concatenating original image Lightness:
lab_output = np.concatenate((img_l[:,:,np.newaxis],result_us),axis=2)
colorized = np.clip(cv2.cvtColor(lab_output, cv2.COLOR_Lab2BGR), 0,1)

# Display and save output colorized images:
cv2.imwrite("Colorized Image.jpg", colorized)
cv2.imshow("Colorized Image", colorized)
cv2.waitKey(0)